In [1]:
import os
from os import walk
import SimpleITK
import SimpleITK as sitk

import numpy as np
import pandas as pd
from pandas import DataFrame
from medpy.io import load, save
import random
import PIL
from PIL import Image
import scipy.ndimage
import json
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline 
plt.rcParams['figure.figsize'] = [10, 10]

import utils
import time
from tqdm import tqdm

config = json.load(open('config.json'))
data_folder_path = config["data_folder_path"]
metadata_folder = f"{data_folder_path}/extended_training_data/metadata/"
mha_folder = f"{data_folder_path}/extended_training_data/images/C0.25/"
png_folder = f"{data_folder_path}/extended_training_data/images/C0.25_after_pngfix/"

In [7]:

image_names = next(walk(mha_folder))[2]
image_names = np.sort(image_names).copy()
old_csv = pd.read_csv(f"{metadata_folder}metadata_before.csv")
old_csv = old_csv.sort_values('img_name', ascending=True)
len(old_csv)
assert np.all(old_csv['img_name'].to_numpy().astype('<U19') == image_names)
np.all(old_csv['img_name'].to_numpy().astype('<U19') == image_names)


True

In [8]:
gen_metadata = {
        'height':    [],
        'img_name':  [],
        'width':     [],
        'x'    :     [],
        'y'    :     [],
        'original_name': [],
        'dataset':   [],
        }

for i, fname in enumerate(image_names):
    # png
    img_array = load(f"{mha_folder}{fname}")[0].T
    img_array = utils.scale(img_array,0,255).astype(np.uint8)
    img = Image.fromarray(img_array).convert("L")
    new_name = f"{fname.split('.')[0]}.png"
    
    #csv
    df = old_csv.iloc[i]
    
    if df['img_name'].split('.')[0] == new_name.split('.')[0]: # check if file in csv original csv is the same as 
    
        gen_metadata['height'].append(df['height'])
        gen_metadata['original_name'].append('-')
        gen_metadata['width'].append(df['width'])
        gen_metadata['x'].append(df['x'])
        gen_metadata['y'].append(df['y'])
        gen_metadata['img_name'].append(f"{df['img_name'].split('.')[0]}.png")
        gen_metadata['dataset'].append('-')
        
        
        img.save(f"{png_folder}{new_name}")
    else:
        print(i)
        print(df['img_name'].split('.')[0])
        print(new_name.split('.')[0])
        break
    

pd.DataFrame(gen_metadata, columns=['height', 'img_name', 'width', 'x', 'y', 'original_name', 'dataset']).to_csv(f"{metadata_folder}/metadata_after.csv",index=False)